<a href="https://colab.research.google.com/github/Crispyhambun/finetune_distilGPT/blob/main/internshiproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM, Trainer, TrainingArguments
)

In [ ]:
from peft import peft_model,PeftConfig,get_peft_model,LoraConfig

In [ ]:
model_path= "distilbert/distilgpt2"

In [ ]:
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("mikasenghaas/wikitext-2")
# DatasetDict({
#     train: Dataset({
#         features: ['text'],
#         num_rows: 17556
#     })
#     validation: Dataset({
#         features: ['text'],
#         num_rows: 1841
#     })
#     test: Dataset({
#         features: ['text'],
#         num_rows: 2183
#     })
# })

README.md:   0%|          | 0.00/493 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.20M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/641k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/713k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17556 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1841 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2183 [00:00<?, ? examples/s]

In [ ]:
from peft import PeftConfig, LoraConfig, get_peft_model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenizer_fun(examples):
    tokenizer.truncation_side = "left"  # Ensure truncation is from the left
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",  # Pad to max_length
        return_tensors="pt"  # Return PyTorch tensors
    )

    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
    required_keys = ['input_ids','attention_mask','labels']
    for key in required_keys:
        if key not in tokenized_inputs or tokenized_inputs[key].nelement() == 0:
            print(f"Warning: Key '{key}' is missing or empty in batch.")
    return tokenized_inputs

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Move mapping with tokenizer_fun to after PEFT model declaration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
)

# apply PEFT model
peft_model = get_peft_model(model, lora_config)

# apply the dataset map:
tokenized_data = dataset.map(tokenizer_fun, batched=True)
tokenized_data

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/17556 [00:00<?, ? examples/s]

Map:   0%|          | 0/1841 [00:00<?, ? examples/s]

Map:   0%|          | 0/2183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 17556
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1841
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2183
    })
})

In [ ]:
# Apply PEFT model
peft_model = get_peft_model(model, lora_config)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",         # Output directory
    evaluation_strategy="epoch",    # Evaluation frequency
    learning_rate=5e-5,             # Learning rate
    per_device_train_batch_size=8,  # Batch size per GPU
    per_device_eval_batch_size=8,   # Batch size for evaluation
    num_train_epochs=3,             # Number of epochs
    weight_decay=0.01,              # Weight decay for regularization
    save_steps=100,                 # Save model every 100 steps
    fp16=True,
    logging_dir='./logs',           # Directory for logs
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install accelerate
from accelerate import Accelerator


In [ ]:
accelerator = Accelerator()


In [ ]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
)

In [ ]:
trainer.train()

# Save the trained model
trainer.save_model("./finetuned_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,9.313700,No log
2,9.219000,No log


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

Epoch,Training Loss,Validation Loss
1,9.313700,No log
2,9.219000,No log
3,9.169700,No log


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

In [ ]:
input_text = input("Enter text: ")
peft_model = peft_model.to(
    device=(
        "cuda"
    )
)
input_ids = tokenizer(str(input_text), return_tensors="pt").input_ids.to(trainer.model.device)
generated_output = peft_model.generate(input_ids, max_length=100)
generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
print(generated_text)